In [1]:
import pandas as pd
import numpy as np

import torch
from torch import nn
from torch.nn import functional as F

In [2]:
wine_data = pd.read_csv("data/wine_preprocessed.csv")
wine_data.head()

,Class,Alcohol,Malic acid,Ash,Alcalinity of ash,Magnesium,Total phenols,Flavanoids,Nonflavanoid phenols,Proanthocyanins,Color intensity,Hue,OD280/OD315 of diluted wines,Proline
0,1,14.23,1.71,2.43,15.6,127,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065
1,1,13.20,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050
2,1,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185
3,1,14.37,1.95,2.50,16.8,113,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480
4,1,13.24,2.59,2.87,21.0,118,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735


In [3]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

wine_data["Class"] = le.fit_transform(wine_data["Class"])

In [4]:
wine_features = wine_data.drop("Class", axis=1)

In [5]:
wine_target = wine_data[["Class"]]
wine_target.sample(5)

,Class
143,2
134,2
140,2
68,1
157,2


In [6]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(wine_features, wine_target, test_size=0.2, random_state=42)

In [7]:
X_train.shape, y_train.shape

((142, 13), (142, 1))

In [8]:
Xtrain_ = torch.from_numpy(X_train.values).float()
Xtest_ = torch.from_numpy(X_test.values).float()

In [9]:
ytrain_ = torch.from_numpy(y_train.values).view(1, -1)[0]
ytest_ = torch.from_numpy(y_test.values).view(1, -1)[0]

In [10]:
input_size = 13
output_size = 3
hidden_size = 100

In [11]:
class StupidNet(nn.Module):
    def __init__(self):
        super(StupidNet, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, output_size)
    
    def forward(self, X):
        X = torch.sigmoid(self.fc1(X))
        X = torch.sigmoid(self.fc2(X))
        X = self.fc3(X)
        return F.log_softmax(X, dim=-1)

In [12]:
epochs = 1000

In [13]:
import os
loss_fn = nn.NLLLoss()
    
def train(model, Xtrain_, ytrain_, optimizer):
    for epoch in range(epochs):
        
        pid = os.getpid()

        optimizer.zero_grad()
        ypred_ = model(Xtrain_)

        loss = loss_fn(ypred_, ytrain_)

        loss.backward()
        optimizer.step()

        if epoch % 100 == 0:
            print("Pid", pid, "Epoch", epoch, "loss", loss.item())

In [14]:
import torch.multiprocessing as mp
mp.get_sharing_strategy()

'file_descriptor'

In [15]:
from sklearn.metrics import accuracy_score

if __name__ == "__main__":
    
    mp.set_start_method("fork")
    
    Xtrain_.share_memory_()
    Xtest_.share_memory_()
    ytrain_.share_memory_()
    ytest_.share_memory_()
    
    model = StupidNet()    
    model.share_memory_()

    import torch.optim as optim
    optimizer = optim.Adam(model.parameters(), lr=0.01)
    
    processes = []
    
    for rank in range(4):
        p = mp.Process(target=train, args=(model, Xtrain_, ytrain_, optimizer))
        p.start()
        processes.append(p)
        
    for p in processes:
        p.join()
        
    model.eval()
    test_loss = 0
    correct = 0
    predict_out = model(Xtest_)
    predict_y = torch.argmax(predict_out, 1)
    print(accuracy_score(ytest_.data, predict_y.data))

AttributeError: 'StupidNet' object has no attribute 'share_memory_'